In [2]:
import os
import pandas as pd


## Imp - importing path

In [3]:
calendar_file_path = os.path.join('..', 'data', 'calendar.csv')
sales_file_path = os.path.join('..', 'data', 'sales_train_validation.csv')
prices_file_path = os.path.join('..', 'data', 'sell_prices.csv')
calendar = pd.read_csv(calendar_file_path)
sales = pd.read_csv(sales_file_path,nrows=1000)
prices = pd.read_csv(prices_file_path)

In [6]:
cat_cols = sales.select_dtypes(include='object').columns
num_cols = sales.select_dtypes(exclude='object').columns

## Important- convert sales df to long format

In [ ]:
sales_long = sales.melt(
    id_vars=["id", "item_id", "dept_id", "cat_id", "store_id", "state_id"],
    var_name="d",
    value_name="sales"
)



,id,item_id,dept_id,cat_id,store_id,state_id,d,sales
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0


## Imp - Merging calendar with sales

In [15]:
test = sales_long.copy()
test = pd.merge(test, 
                calendar[['d', 'date', 'event_name_1','wm_yr_wk', 'snap_CA', 'snap_TX', 'snap_WI']], 
                on='d', 
                how='left')
test["snap"] = test.apply(lambda row: row["snap_" + row["state_id"]], axis=1)
test.drop(columns=['snap_CA', 'snap_TX', 'snap_WI'
                   ], inplace=True)
test.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,event_name_1,wm_yr_wk,snap
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,NaN,11101,0
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,NaN,11101,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,NaN,11101,0
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,NaN,11101,0
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,NaN,11101,0


## imp - merging new df with sell_prices.csv

In [17]:
t = test.copy()
t = pd.merge(test,prices, on=['store_id', 'item_id', 'wm_yr_wk'], how='left')
t.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,event_name_1,wm_yr_wk,snap,sell_price
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,NaN,11101,0,NaN
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,NaN,11101,0,NaN
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,NaN,11101,0,NaN
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,NaN,11101,0,NaN
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,NaN,11101,0,NaN


## Imp - fix missing sell prices by forward and backward filling

In [ ]:
# Fill missing prices with the last known price within each store & item
t['sell_price'] = t.groupby(['store_id', 'item_id'])['sell_price'].ffill()
t['sell_price'] = t.groupby(['store_id', 'item_id'])['sell_price'].bfill()


,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,event_name_1,wm_yr_wk,snap,sell_price
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,NaN,11101,0,9.58
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,NaN,11101,0,3.97
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,NaN,11101,0,2.97
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,NaN,11101,0,4.34
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,NaN,11101,0,2.98


In [22]:
t.drop(columns=[ 'id', 'wm_yr_wk','d'], inplace=True)
t.head()



,item_id,dept_id,cat_id,store_id,state_id,sales,date,event_name_1,snap,sell_price
0,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,2011-01-29,NaN,0,9.58
1,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,2011-01-29,NaN,0,3.97
2,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,2011-01-29,NaN,0,2.97
3,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,2011-01-29,NaN,0,4.34
4,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,2011-01-29,NaN,0,2.98
